In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup as bs
import csv
import pyautogui

In [10]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyautogui
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active
ws.append(['도서명','가격','링크'])

service = Service(excutable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(5)
driver.maximize_window()

# 웹사이트 열기 (yes 24로 이동)
driver.get('http://www.yes24.com/main/default.aspx')
driver.implicitly_wait(2)
time.sleep(2)

# 도서 검색창 클릭
search = driver.find_element(By.CSS_SELECTOR, '#query')
search.click()

# 검색어 입력
keyword = pyautogui.prompt('검색어를 입력하세요.')
lastpage = pyautogui.prompt('페이지를 입력하세요.')

# 검색어 전송, 엔터키 누르기
search.send_keys(f'{keyword}')
search.send_keys(Keys.ENTER)

# 입력받은 페이지만큼 for문 돌리기

for i in range(1, int(lastpage)+1):
    print(f'===== {i}페이지=====')
    driver.implicitly_wait(4)
    
    url = f'http://www.yes24.com/Product/Search?domain=ALL&query={keyword}&page={i}'
    driver.get(url)
    time.sleep(2)
        
    # 영역 잡기    
    items = driver.find_elements(By.CSS_SELECTOR, 'div.itemUnit')
    
    
    # 2페이지부터 오류가 발생하기 때문에 try문으로 정리
    try:
        for item in items:
            j = item.find_element(By.CSS_SELECTOR, 'span.gd_res').text
            # 영역에 중고도서까지 포함되어 있기 때문에
            # 그냥 도서인것만 가져오기
            if '중고도서' not in j:
                    name = item.find_element(By.CSS_SELECTOR, 'a.gd_name').text
                    price = item.find_element(By.CSS_SELECTOR, 'div.info_row.info_price > strong > em').text
                    link = item.find_element(By.CSS_SELECTOR, 'a.gd_name').get_attribute('href')
                    
                    print(f'도서명 : {name} \n 가격 : {price}원 \n {link}')
                    ws.append([name, price, link])
    except:
        pass

# 엑셀 저장
wb.save(f'./data/crawling/{keyword}.xlsx')
pyautogui.alert('완료!')

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.80M/6.80M [00:00<00:00, 57.5MB/s]


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'